In [49]:
"""Data processing usando python para problema DMEyF"""

import warnings
from pathlib import Path


import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier

# La carpeta data debe tener el csv competencia1_2022.csv
data = pd.read_csv(Path("data", "competencia1_2022.csv"))
# La carpeta data debe tener el csv DiccionarioDatos.ods
diccionario = pd.read_excel(Path("data", "DiccionarioDatos.ods"), engine='odf')
# Guardamos el target en la variable target ("y" a predecir)
target = data.pop("clase_ternaria")
# Esta serie la usaremos para agregarla a data que fue spliteada en el processing
numero_cliente = data.pop("numero_de_cliente")

C:\Users\andre\AppData\Local\Temp\ipykernel_6520\1102417259.py:13: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(Path("data", "competencia1_2022.csv"))


In [50]:
data.head()

,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,202101,1,0,0,56,132,1897.12,24324.75,940.32,-490.67,...,0.0,4.0,0.0,-16247.77,0.00,3997.0,18435.59,1.0,0.0,1337.22
1,202101,1,0,0,48,100,-384.72,33860.45,-515.53,-19.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202101,1,0,0,60,53,-3622.66,-21310.32,2442.04,-5747.04,...,0.0,4.0,0.0,-33430.50,0.00,1573.0,12033.40,4.0,0.0,20011.38
3,202101,1,0,0,46,273,14542.05,125727.99,1037.77,9338.64,...,0.0,4.0,0.0,-9832.94,14.27,2063.0,758.21,3.0,0.0,1302.03
4,202101,1,0,0,47,192,1868.27,37305.29,1496.68,-11.71,...,0.0,4.0,0.0,0.00,0.00,5842.0,6316.13,1.0,0.0,4680.27


# Hacemos un dataset simple de eliminar los NaNs y eliminar variables que no sean detectadas como importantes.

In [51]:
# Eliminamos todas las columnas que tengan al menos un NaN
data_short = data.dropna(axis = 1)
# Dejamos la data de training 202101
data_train = data_short[data_short.foto_mes == 202101]
target = target.loc[data_train.index]
data_kaggle = data_short[data_short.foto_mes == 202103]

In [52]:
# Entrenamos un árbol muy simple con parámetros que resultaron 'buenos' vistos en clase
model = DecisionTreeClassifier(max_depth = 5, min_samples_split = 1073, ccp_alpha = 0, )
model.fit(X = data_train, y = target)

DecisionTreeClassifier(ccp_alpha=0, max_depth=5, min_samples_split=1073)

# Vamos a calcular la importancia de cada uno de los atributos

In [53]:
importance = pd.Series(model.feature_importances_, index=data_train.columns)
importance.sort_values(ascending = False)

ctrx_quarter                0.468661
mcuentas_saldo              0.126352
cdescubierto_preacordado    0.106432
cprestamos_personales       0.069993
mcaja_ahorro                0.068424
                              ...   
cinversion1                 0.000000
mplazo_fijo_pesos           0.000000
cplazo_fijo                 0.000000
mprestamos_hipotecarios     0.000000
cmobile_app_trx             0.000000
Length: 107, dtype: float64

### Eliminación de variables que *no aportan*

In [54]:
# Eliminamos las feautures que no aportan
def eliminate_features(data: pd.DataFrame, importance: pd.Series, tol: float):
    unuseful = importance[importance <= tol]
    return data[[col for col in data.columns if col not in unuseful.index]]

data_train = eliminate_features(data_train, importance, tol = 0.01)
data_kaggle = eliminate_features(data_kaggle, importance, tol = 0.01)

In [55]:
data_train.head()

,mpasivos_margen,mcaja_ahorro,cdescubierto_preacordado,mcuentas_saldo,mtarjeta_visa_consumo,cprestamos_personales,mprestamos_personales,cprestamos_hipotecarios,mplazo_fijo_dolares,cseguro_auto,mcomisiones_mantenimiento,ctrx_quarter
0,1306.36,85151.91,1,125822.25,18435.59,0,0.00,0,0.0,0,0.00,71
1,131.34,9851.83,0,331.99,0.00,1,139299.38,0,0.0,0,0.00,64
2,576.78,11755.80,1,85698.96,19580.78,0,0.00,0,0.0,0,1622.81,75
3,2173.96,160517.29,1,258855.68,7713.08,0,0.00,1,0.0,0,0.00,70
4,328.49,23043.96,1,10618.20,6323.25,0,0.00,0,0.0,0,1364.94,59


In [56]:
data_kaggle.head()

,mpasivos_margen,mcaja_ahorro,cdescubierto_preacordado,mcuentas_saldo,mtarjeta_visa_consumo,cprestamos_personales,mprestamos_personales,cprestamos_hipotecarios,mplazo_fijo_dolares,cseguro_auto,mcomisiones_mantenimiento,ctrx_quarter
161342,979.55,65150.26,1,24675.22,15732.34,0,0.00,0,0.0,0,0.00,64
161343,149.29,11234.24,0,41.62,0.00,1,136790.07,0,0.0,0,0.00,65
161344,348.42,12388.44,1,2635.47,14376.18,0,0.00,0,0.0,0,1622.81,65
161345,2068.16,125222.13,1,314336.17,16085.33,0,0.00,1,0.0,0,0.00,64
161346,179.80,10172.40,1,18253.96,1286.93,0,0.00,0,0.0,0,1774.04,62


### Guardamos ese csv para ser Ejecutado con el algoritmo de rpart

In [57]:
def make_data_to_r(data_train, data_kaggle):
    # Concateneamos la data
    data_to_r = pd.concat([data_train, data_kaggle], axis = 0)
    # Agregamos la columna target (que tendrá nan para la data de 202103)
    data_to_r["clase_ternaria"] = target
    # Agregamos el número de cliente
    data_to_r["numero_de_cliente"] = numero_cliente
    return data_to_r

data_to_r = make_data_to_r(data_train, data_kaggle)

In [58]:
data_to_r.head()

,mpasivos_margen,mcaja_ahorro,cdescubierto_preacordado,mcuentas_saldo,mtarjeta_visa_consumo,cprestamos_personales,mprestamos_personales,cprestamos_hipotecarios,mplazo_fijo_dolares,cseguro_auto,mcomisiones_mantenimiento,ctrx_quarter,clase_ternaria,numero_de_cliente
0,1306.36,85151.91,1,125822.25,18435.59,0,0.00,0,0.0,0,0.00,71,CONTINUA,31115668
1,131.34,9851.83,0,331.99,0.00,1,139299.38,0,0.0,0,0.00,64,CONTINUA,31115678
2,576.78,11755.80,1,85698.96,19580.78,0,0.00,0,0.0,0,1622.81,75,CONTINUA,31117415
3,2173.96,160517.29,1,258855.68,7713.08,0,0.00,1,0.0,0,0.00,70,CONTINUA,31117711
4,328.49,23043.96,1,10618.20,6323.25,0,0.00,0,0.0,0,1364.94,59,CONTINUA,31118145


In [59]:
data_to_r.to_csv("data_r_dropnan.csv", index = False)

# Hacemos un dataset inputando missings usando KNN inputer con N = 1

Usamos ``K = 1`` pues el inputer por defecto toma la media aritmetica para las variables missing de los ``K`` vecinos encontrados, al setear ``K = 1``, simplemente se copia el missing del vecino encontrado, lo que hace sentido en variables binarias y no continuas. Una aplicación de un inputer más sofisticado debe ser implementada.




In [35]:
# Volvemos a separar desde la data original el conjunto de training, que ahora en este punto contiene los nana
data_train = data[data.foto_mes == 202101]
data_kaggle = data[data.foto_mes == 202103]
# Usamos KNN con K = 1
inputer = KNNImputer(n_neighbors = 1)
# Hacemos el fit-transform sobre train
data_train = inputer.fit_transform(data_train)
# Hacemos el transform sobre test
data_kaggle = inputer.transform(data_kaggle)

In [ ]:
# Creamos nuestra data para R
data_to_r = make_data_to_r(data_train, data_kaggle)
data_to_r.to_csv("data_r_knn.csv", index = False)

# Construcción de nuevas features a partir de árbol (No fue usado en kaggle, pero en los cross validations con los algoritmos de R no performó para nada bien) 

### Construcción de features a partir de un árbol sin usar ``ctrx_quarter`` 

In [48]:
def create_new_variables_from_tree(
    tree: DecisionTreeClassifier, X: pd.DataFrame, leaves_indexes: set
) -> pd.DataFrame:
    """Crea nuevas variables desde un árbol

    Cuando se entrena un modelo en de árbol en sklearn, este posee un
    atributo ``tree_`` que contiene información del grafo que
    se creo para el árbol. Cada hoja (tanto terminales como internas)
    queda codificada con índice número desde 0 a ``NUMERO_TOTAL_HOJAS``.
    El párametro ``leaves_indexes`` corresponden a los índices de las
    hojas *terminales*. No he encontrado el método directo para
    sacarlo como atributo desde ``tree``, se puede hacer a la mala
    haciendo ``set(tree.apply(data_train))``, que retorna los índices
    de los nodos terminales y donde data_train corresponde al conjunto
    para el cual se entreno el árbol ``tree``. ```tree.apply(X)``
    es un array de largo igual a la cantidad de filas de ``X`` cuyos
    valores son el índice del nodo terminal.

    En este contexto ``X`` es una dataframe para el cual se crearan
    las nuevas variables provenientes del árbol tree (``X`` será
    tanto el conjunto de entrenamiento como la data de testeo que irá
    a kaggle)

    """

    def build_new_vars(row, model, leaves_indexes):
        leaves_indexes = {real_index: i for i, real_index in enumerate(leaves_indexes)}
        n_leaves = len(leaves_indexes)
        index = leaves_indexes[model.apply(row.to_numpy().reshape(1, -1))[0]]
        return pd.Series([0 if idx != index else 1 for idx in range(n_leaves)])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        new_vars = X.apply(
            lambda row: build_new_vars(row, model=tree, leaves_indexes=leaves_indexes),
            axis=1,
        )
    new_vars = new_vars.rename(
        columns={idx: f"New_var_{idx}" for idx in range(len(leaves_indexes))}
    )
    return new_vars

In [61]:
data_train = data_short[data_short.foto_mes == 202101]
data_kaggle = data_short[data_short.foto_mes == 202103]
data_train_without_ctrx_quarter = data_train.drop(columns = "ctrx_quarter")
data_kaggle_without_ctrx_quarter = data_kaggle.drop(columns = "ctrx_quarter")
model_without_ctrx_quarter = DecisionTreeClassifier(max_depth = 8, min_samples_split = 5, ccp_alpha = 0, )
model_without_ctrx_quarter.fit(X = data_train_without_ctrx_quarter, y = target)

DecisionTreeClassifier(ccp_alpha=0, max_depth=8, min_samples_split=5)

In [62]:
# Los indices de los nodos terminales (como se podrá sacar de otra forma? sin pasar por la evaluacion)
LEAVES_INDEXES = set(model_without_ctrx_quarter.apply(data_train_without_ctrx_quarter))

In [63]:
train_new_vars = create_new_variables_from_tree(
    model_without_ctrx_quarter, 
    data_train_without_ctrx_quarter, 
    LEAVES_INDEXES
)

kaggle_new_vars = create_new_variables_from_tree(
    model_without_ctrx_quarter, 
    data_kaggle_without_ctrx_quarter, 
    LEAVES_INDEXES
)

In [64]:
train_new_vars.head()

,New_var_0,New_var_1,New_var_2,New_var_3,New_var_4,New_var_5,New_var_6,New_var_7,New_var_8,New_var_9,...,New_var_104,New_var_105,New_var_106,New_var_107,New_var_108,New_var_109,New_var_110,New_var_111,New_var_112,New_var_113
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
data_train = pd.concat([data_train, train_new_vars], axis = 1)
data_kaggle = pd.concat([data_kaggle, kaggle_new_vars], axis = 1)

In [66]:
# Creamos nuestra data para R
data_to_r = make_data_to_r(data_train, data_kaggle)
data_to_r.to_csv("data_r_new_features.csv", index = False)

In [67]:
data_to_r.head()

,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,...,New_var_106,New_var_107,New_var_108,New_var_109,New_var_110,New_var_111,New_var_112,New_var_113,clase_ternaria,numero_de_cliente
0,202101,1,0,0,56,132,1897.12,24324.75,940.32,-490.67,...,1,0,0,0,0,0,0,0,CONTINUA,31115668
1,202101,1,0,0,48,100,-384.72,33860.45,-515.53,-19.82,...,0,0,0,0,0,0,0,0,CONTINUA,31115678
2,202101,1,0,0,60,53,-3622.66,-21310.32,2442.04,-5747.04,...,0,0,0,0,0,0,0,0,CONTINUA,31117415
3,202101,1,0,0,46,273,14542.05,125727.99,1037.77,9338.64,...,1,0,0,0,0,0,0,0,CONTINUA,31117711
4,202101,1,0,0,47,192,1868.27,37305.29,1496.68,-11.71,...,0,0,0,0,0,0,0,0,CONTINUA,31118145
